In [7]:
import numpy as np
import cv2
import plotly.express as px
import os

In [8]:

img = cv2.imread("data/img01.jpg")
px.imshow(img)

patternSize = (8, 5)

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

retval, corners = cv2.findChessboardCorners(gray, patternSize)

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
ref_zone = (11,11)
zero_zone = (-1, -1)

ref_corners = cv2.cornerSubPix(gray, corners, ref_zone, zero_zone, criteria)
with_corners = cv2.drawChessboardCorners(img, patternSize, corners, retval) 
cv2.imwrite("output/with_corners.jpg", with_corners)

True

In [9]:
def gen_objpoints(num_cols, num_rows, l):
    a, b = np.mgrid[0:num_rows, 0:num_cols]
    a = a.reshape(num_cols, num_rows, 1)
    b = b.reshape(num_cols, num_rows, 1)
    
    a = a * np.array([0, 1, 0])
    b = b * np.array([1, 0, 0])

    grid = a + b
    grid = grid.reshape(-1, 3)
    return np.array(grid * l, dtype=np.float32) 

gen_objpoints(8, 5, 30)

array([[  0.,   0.,   0.],
       [ 30.,   0.,   0.],
       [ 60.,   0.,   0.],
       [ 90.,   0.,   0.],
       [120.,   0.,   0.],
       [150.,   0.,   0.],
       [180.,   0.,   0.],
       [210.,   0.,   0.],
       [  0.,  30.,   0.],
       [ 30.,  30.,   0.],
       [ 60.,  30.,   0.],
       [ 90.,  30.,   0.],
       [120.,  30.,   0.],
       [150.,  30.,   0.],
       [180.,  30.,   0.],
       [210.,  30.,   0.],
       [  0.,  60.,   0.],
       [ 30.,  60.,   0.],
       [ 60.,  60.,   0.],
       [ 90.,  60.,   0.],
       [120.,  60.,   0.],
       [150.,  60.,   0.],
       [180.,  60.,   0.],
       [210.,  60.,   0.],
       [  0.,  90.,   0.],
       [ 30.,  90.,   0.],
       [ 60.,  90.,   0.],
       [ 90.,  90.,   0.],
       [120.,  90.,   0.],
       [150.,  90.,   0.],
       [180.,  90.,   0.],
       [210.,  90.,   0.],
       [  0., 120.,   0.],
       [ 30., 120.,   0.],
       [ 60., 120.,   0.],
       [ 90., 120.,   0.],
       [120., 120.,   0.],
 

In [10]:
cameraMatrix = None
distCoeffs = None

objectPoints = gen_objpoints(8, 5, 30)

imagePoints = ref_corners
imageSize = (1440, 960)
retval, cameraMatrix, distCoeffs, rvecs, tvecs = cv2.calibrateCamera([objectPoints], [imagePoints], imageSize, cameraMatrix, distCoeffs)

R = np.eye(3)
m1type=cv2.CV_32FC1
interpolation=cv2.INTER_LINEAR

optimalCameraMatrix, _ = cv2.getOptimalNewCameraMatrix(cameraMatrix, distCoeffs, imageSize, 1)
map1, map2 = cv2.initUndistortRectifyMap(cameraMatrix, distCoeffs, R, optimalCameraMatrix, imageSize, m1type)
undistorted = cv2.remap(with_corners, map1, map2, interpolation)
cv2.imwrite("output/undistorted.jpg", undistorted)

True

In [13]:
patternSize = (8, 5)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
ref_zone = (11,11)
zero_zone = (-1, -1)

def getFilePoints(filename):
    return getImagePoints(cv2.imread(f"data/{filename}"))

def getImagePoints(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    retval, corners = cv2.findChessboardCorners(gray, patternSize)
    
    ref_corners = cv2.cornerSubPix(gray, corners, ref_zone, zero_zone, criteria)
    return ref_corners

def getImagePointsFromDir(dirname):
    ret = []
    for filename in os.listdir(dirname):
        ret.append((filename, getFilePoints(filename)))
    return ret

objectPoints = gen_objpoints(8, 5, 30)

def getCameraMatrix(pointss):
    cameraMatrix = None
    distCoeffs = None

    _, cameraMatrix, distCoeffs, _, _ = cv2.calibrateCamera(len(pointss)*[objectPoints], pointss, imageSize, cameraMatrix, distCoeffs)

    optimalCameraMatrix, _ = cv2.getOptimalNewCameraMatrix(cameraMatrix, distCoeffs, imageSize, 1)
    return cameraMatrix, optimalCameraMatrix, distCoeffs

R = np.eye(3)
m1type=cv2.CV_32FC1
interpolation=cv2.INTER_LINEAR

def getWithCorners(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    retval, corners = cv2.findChessboardCorners(gray, patternSize)

    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    ref_zone = (11,11)
    zero_zone = (-1, -1)

    ref_corners = cv2.cornerSubPix(gray, corners, ref_zone, zero_zone, criteria)
    return cv2.drawChessboardCorners(img, patternSize, ref_corners, retval)

def getUndistorted(name, map1, map2):
    img = cv2.imread(f"data/{name}")
    return cv2.remap(img, map1, map2, interpolation)

def getUndistortedWithCorners(name, map1, map2):
    img = cv2.imread(f"data/{name}")
    with_corners = getWithCorners(img)
    return cv2.remap(with_corners, map1, map2, interpolation)

def saveUndistortedWithCorners(name, map1, map2):
    undistorted = getUndistortedWithCorners(name, map1, map2)
    cv2.imwrite(f"output/undistorted/{name}", undistorted)


namesAndPoints = list(getImagePointsFromDir("data"))
names, pointss = list(map(list, zip(*namesAndPoints)))
cameraMatrix, optimalCameraMatrix, distCoeffs = getCameraMatrix(pointss)

map1, map2 = cv2.initUndistortRectifyMap(cameraMatrix, distCoeffs, R, optimalCameraMatrix, imageSize, m1type)

for filename, points in namesAndPoints:
    saveUndistortedWithCorners(filename, map1, map2)

def coordinates(point):
    return [int (i) for i in tuple(point.ravel())]

def draw(img, corners, imgpts):
    # WARNING: openCV uses BGR color space
    corner = coordinates(corners[0].ravel())
    img = cv2.line(img, corner, coordinates(imgpts[0]), (0,0,255), 5)
    img = cv2.line(img, corner, coordinates(imgpts[1]), (0,255,0), 5)
    img = cv2.line(img, corner, coordinates(imgpts[2]), (255,0,0), 5)
    return img

def saveWithAxis(filename):
    img = getUndistorted(filename, map1, map2)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    retval, corners = cv2.findChessboardCorners(gray, patternSize)
    retval, rvec, tvec = cv2.solvePnP(objectPoints, corners, cameraMatrix, distCoeffs)

    axisPoints = np.array([(90, 0, 0), (0, 90, 0), (0, 0, -90)], dtype=np.float32)
    imagePoints, jacobian = cv2.projectPoints(axisPoints, rvec, tvec, cameraMatrix, distCoeffs)
    with_corners = draw(img, corners, imagePoints)
    cv2.imwrite(f"output/with_corners_after_undistort/{filename}", with_corners)



for filename, points in namesAndPoints:
    saveWithAxis(filename)

In [ ]:
def get_ransac_params(poll_size = 4, poll_count = 100, epsilon = 5):
    return {
        "poll_size": poll_size,
        "poll_count": poll_count,
        "epsilon": epsilon
    }

filename = "img01.jpg"

image_points = getFilePoints(filename)

print(image_points.shape)

noise = np.random.normal(0, 30, image_points.shape)

img = cv2.imread("data/img01.jpg")

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

retval, corners = cv2.findChessboardCorners(gray, patternSize)

shape = corners.shape

noise = np.random.normal(0, 30, corners.shape)

shape[-1] = 1

mask = np.random.rand(*shape) >= 0.5



def ransac(poll_size = 4, poll_count = 100, epsilon = 5):
    best_fit_count = 0
    best_fit = None
    for k in range(poll_count):
        


(40, 1, 2)
